In [231]:
#importing
import numpy as np
import pandas as pd
import multiprocessing
import math 
import xgboost as xgb
from numpy import mean
from statistics import variance
from numpy import std
from pandas import read_csv
from sklearn.impute import KNNImputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error as mae
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostRegressor
import warnings

In [232]:
#warnings.filterwarnings('ignore')

In [233]:
multiprocessing.cpu_count()

8

In [234]:
data_wd = "C:/Users/Lenovo/Desktop/git/STAT4011_Project1/data/Lianjia_transformed.csv"
df = pd.read_csv(data_wd)

In [235]:
X = df.drop(columns=['price'])
y = df['price']
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=4011)

# Define a model Assess Function

In [236]:
def model_assess(tuned_model,X_train,y_train,X_test,y_test):
    y_train_pred=tuned_model.predict(X_train)
    y_test_pred=tuned_model.predict(X_test)
    train_R2 = tuned_model.score(X_train, y_train)
    test_R2 = tuned_model.score(X_test, y_test)
    train_RMSE=round(math.sqrt(mean_squared_error(y_train_pred,y_train)),4)
    test_RMSE=round(math.sqrt(mean_squared_error(y_test_pred,y_test)),4)
    train_MAE=round(mae(y_train_pred,ytrain),4)
    test_MAE=round(mae(y_test_pred,ytest),4)
    return([train_R2,test_R2,train_RMSE,test_RMSE,train_MAE,test_MAE])

# OLS

In [237]:
model_ols = LinearRegression().fit(Xtrain, ytrain)

In [238]:
ols_result=model_assess(model_ols,Xtrain,ytrain,Xtest,ytest)

# LASSO

In [239]:
# step 1: examine the model
model_lasso = LassoCV(cv=5, random_state=4011,n_jobs=-1,alphas=np.logspace(-5,-1,100),tol=1e-2).fit(Xtrain, ytrain)

In [240]:
# step 2: examine if the selected hyperparameter lies within boundary
model_lasso.alpha_ 

0.0038535285937105314

In [241]:
# step 3: model assessment
lasso_result=model_assess(model_lasso,Xtrain,ytrain,Xtest,ytest)

In [242]:
# step 4: examine if coefficients are shrinked towards 0; this is only for lasso and ridge
print(list(zip(model_lasso.coef_, X))) 

[(6.746396362533325, 'total_floor'), (26.433482136701418, 'area_size'), (3.7420521258645536, 'elevator'), (-0.42096604583414676, 'date_on_market'), (0.34081364442938944, 'last_transact_date'), (1.1199652772022928, 'bedrm_no'), (0.9128766955498774, 'parlour_no'), (0.048710482182980686, 'kitchen_no'), (3.94072790244851, 'toilet_no'), (-3.904172401406128, 'stair'), (2.142749096850644, 'unit'), (4.636422005474458, 'proportion'), (0.8743437420303899, 'mid_floor'), (0.08073072934462963, 'higher_floor'), (-0.8475476505669389, 'duplex_config'), (0.46740503400994804, 'levelling_config'), (0.2150233561903785, 'split_level_config'), (0.19744061750489922, 'diepin'), (-2.1963970291601607, 'talou'), (0.25537760298219936, 'pingfang'), (0.2947684186077513, 'bantajiehe'), (1.3968058560207328, 'banlou'), (2.153574876872985, 'dudong'), (-1.3155647517181457, 'lianpai'), (0.44731355258225675, 'south'), (-0.1443868466026778, 'west'), (-0.17237575966748908, 'north'), (-0.4619997462640114, 'northeast'), (0.23

# RIDGE

In [243]:
model_ridge = RidgeCV(cv=5,alphas=np.logspace(-1,3,100)).fit(Xtrain, ytrain)
model_ridge.alpha_ 

81.11308307896873

In [244]:
ridge_result=model_assess(model_ridge,Xtrain,ytrain,Xtest,ytest)

In [245]:
#examine if coefficients are shrinked 
print(list(zip(model_ridge.coef_, X))) 

[(6.706065953939077, 'total_floor'), (25.97101465070193, 'area_size'), (3.7343712781167637, 'elevator'), (-0.4083998722356938, 'date_on_market'), (0.360552376120507, 'last_transact_date'), (1.292567365242712, 'bedrm_no'), (0.9637350515243888, 'parlour_no'), (0.048777130026567236, 'kitchen_no'), (4.082070884338945, 'toilet_no'), (-3.816967999201759, 'stair'), (2.0351392702175866, 'unit'), (4.606564294168615, 'proportion'), (0.858490799554046, 'mid_floor'), (0.06009602990876299, 'higher_floor'), (-0.8212047976799733, 'duplex_config'), (0.45694061694912474, 'levelling_config'), (0.21755780014733972, 'split_level_config'), (0.3012927271798949, 'diepin'), (-2.0723541450762886, 'talou'), (0.26495266367398296, 'pingfang'), (0.3470947386969166, 'bantajiehe'), (1.490416283581285, 'banlou'), (2.3140648088912186, 'dudong'), (-1.1473689306726822, 'lianpai'), (0.4612726397069366, 'south'), (-0.13667745363910186, 'west'), (-0.18127108084585633, 'north'), (-0.4587489845465758, 'northeast'), (0.239388

# Elastic Net

In [246]:
model_elanet = ElasticNetCV(cv=5,l1_ratio=[0.5,0.7,0.8,0.9,0.95,1],alphas=np.logspace(-10,-1,20),n_jobs=-1,tol=1e-1,max_iter=1e5).fit(Xtrain, ytrain)

In [247]:
model_elanet.alpha_

5.455594781168514e-06

In [248]:
model_elanet.l1_ratio_ 

1.0

In [249]:
elanet_result=model_assess(model_elanet,Xtrain,ytrain,Xtest,ytest)

# KNN Regression

In [252]:
param_grid_knr = {'n_neighbors': [1,2,3,4], 'weights': ['uniform', 'distance']}

select_model_knr = GridSearchCV(KNeighborsRegressor(),param_grid_knr, n_jobs=-1, cv=5, 
                                scoring='neg_mean_squared_error', return_train_score=True)
select_model_knr.fit(Xtrain, ytrain)
model_knr = select_model_knr.best_estimator_
select_model_knr.best_params_

{'n_neighbors': 1, 'weights': 'uniform'}

In [253]:
knr_result = model_assess(model_knr, Xtrain,ytrain,Xtest,ytest)
knr_result

[1.0, 0.6751196094991677, 0.0, 26.2494, 0.0, 14.2367]

# SVR

In [254]:
param_grid_svr = [
    {'kernel': ["linear","poly","rbf","sigmoid"]}]

select_model_svr = GridSearchCV(SVR(),param_grid_svr, n_jobs=-1, cv=5,scoring='neg_mean_squared_error',
return_train_score=True)
select_model_svr.fit(Xtrain, ytrain)

model_svr = select_model_svr.best_estimator_
select_model_svr.best_params_

{'kernel': 'linear'}

In [255]:
svr_result= model_assess(model_svr,Xtrain,ytrain,Xtest,ytest)
svr_result 

[0.7629479994613417, 0.7505080810764413, 20.749, 23.0031, 11.2143, 11.6821]

# Decision Tree

In [264]:
param_grid_tree = {'max_depth':[15,20,25,30], 'min_samples_split': [1,2,3,4], 
 'min_samples_leaf': [3,4,5,6], 'max_features': [40,50,60,70], 'max_leaf_nodes': [350,400,450,500]}

tree = DecisionTreeRegressor(random_state=4011)
select_model_tree = GridSearchCV(tree, param_grid_tree, n_jobs=-1, cv=5,
scoring='neg_mean_squared_error',
return_train_score=True).fit(Xtrain, ytrain)

model_tree_reg = select_model_tree.best_estimator_
select_model_tree.best_params_

{'max_depth': 20,
 'max_features': 60,
 'max_leaf_nodes': 350,
 'min_samples_leaf': 5,
 'min_samples_split': 2}

In [265]:
tree_reg_result = model_assess(model_tree_reg,Xtrain,ytrain,Xtest,ytest)
tree_reg_result

[0.9023304878249709, 0.7124068851082368, 13.3185, 24.6972, 8.0439, 10.4981]

# Random Forest

In [267]:
param_grid_rfr = {'max_features': [40,50,60,70],'n_estimators': [34,38,42,46]}

forest_reg = RandomForestRegressor(random_state=4011,max_depth=15)
select_model_rfr = GridSearchCV(forest_reg, param_grid_rfr, n_jobs=-1, cv=5,
scoring='neg_mean_squared_error',
return_train_score=True).fit(Xtrain, ytrain)

model_rfr = select_model_rfr.best_estimator_
select_model_rfr.best_params_

{'max_features': 60, 'n_estimators': 42}

In [268]:
rfr_result = model_assess(model_rfr,Xtrain,ytrain,Xtest,ytest)
rfr_result

[0.9652926377198563, 0.8377568905317605, 7.9394, 18.5499, 4.7664, 8.4334]

# Tree with AdaBoost

In [279]:
param_grid_ada = [
    {'learning_rate': np.logspace(-2,1,10),'n_estimators': [60,70,80,90]}
]

In [ ]:
ada_reg=AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=20),random_state=4011)

select_model_ada = GridSearchCV(ada_reg, param_grid_ada, n_jobs=-1, cv=5,
scoring='neg_mean_squared_error',
return_train_score=True).fit(Xtrain, ytrain)

model_ada = select_model_ada.best_estimator_
select_model_ada.best_params_

In [ ]:
ada_result = model_assess(model_ada,Xtrain,ytrain,Xtest,ytest)
ada_result

# Tree with XGB boosting

In [168]:
param_grid_xgb = [
    {"booster":["gbtree"],"objective":["reg:squarederror"],'learning_rate': np.logspace(-2,0,10),'gamma': np.logspace(-5,0,5),
                'n_estimators': [30,34,38]}
]

In [169]:
XGB_reg=xgb.XGBRegressor(max_depth=15)

select_model_XGB = GridSearchCV(XGB_reg, param_grid_xgb, n_jobs=-1, cv=5,
scoring='neg_mean_squared_error',
return_train_score=True).fit(Xtrain, ytrain)

model_XGB = select_model_XGB.best_estimator_
select_model_XGB.best_params_

{'booster': 'gbtree',
 'gamma': 0.01,
 'learning_rate': 0.3593813663804626,
 'max_depth': 15,
 'objective': 'reg:squarederror'}

In [170]:
XGB_result = model_assess(model_XGB,Xtrain,ytrain,Xtest,ytest)
XGB_result

[0.989116076500549, 0.81003088601129, 4.446, 20.0724, 2.7423, 8.5473]

# Save All Results

In [275]:
all_results=np.vstack([ols_result,lasso_result,ridge_result,elanet_result,knr_result,svr_result,
                      tree_reg_result,rfr_result,ada_result,XGB_result])

In [276]:
all_results=pd.DataFrame(all_results)
all_results.columns=["train_R2","test_R2","train_RMSE","test_RMSE","train_MAE","test_MAE"]
all_results.index=["ols","lasso","ridge","elastic_net","knn","svr","tree","rf","ada","xgb"]

In [277]:
all_results

,train_R2,test_R2,train_RMSE,test_RMSE,train_MAE,test_MAE
ols,0.792032,0.773265,19.4346,21.9289,12.0421,12.5958
lasso,0.792025,0.773246,19.4349,21.9298,12.0369,12.5899
ridge,0.791957,0.772686,19.4380,21.9569,12.0241,12.5830
elastic_net,0.792032,0.773264,19.4346,21.9289,12.0421,12.5958
knn,1.000000,0.675120,0.0000,26.2494,0.0000,14.2367
svr,0.762948,0.750508,20.7490,23.0031,11.2143,11.6821
tree,0.902330,0.712407,13.3185,24.6972,8.0439,10.4981
rf,0.965293,0.837757,7.9394,18.5499,4.7664,8.4334
ada,0.997659,0.850397,2.0618,17.8126,1.0785,7.8214
xgb,0.989116,0.810031,4.4460,20.0724,2.7423,8.5473


In [278]:
all_results.to_csv("C:/Users/Lenovo/Desktop/git/STAT4011_Project1/data/model_results.csv",index=True)